In [2]:
import os
from lxml import etree
import lxml.html
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from user_agent import generate_user_agent

import requests
from tqdm import tqdm
import time
import json
import random

import pandas as pd

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
co = webdriver.ChromeOptions()
# co.add_argument('--headless')  # 无界面模式
co.add_argument('--disable-images')
# co.add_argument('--disable-javascript') #禁用js
# co.add_argument('--proxy-server=http://ip:port') #使用代理
co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))

# 自动安装 webdriver

自动安装webdriver参见：https://stackoverflow.com/questions/60296873/sessionnotcreatedexception-message-session-not-created-this-version-of-chrome/62127806

In [5]:
browser = webdriver.Chrome(executable_path=ChromeDriverManager().install(),options=co,service_log_path=os.path.devnull)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/taptap/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


# 抓取租房贴

需要登录，否则大概没几页就提示登录了

In [6]:
url = 'https://www.douban.com/group/shanghaizufang/discussion?start=0'

browser.get(url)

In [29]:
def get_topics(url):
    
    browser.get(url)

    html = browser.page_source

    s = etree.HTML(html)

    url_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="title"]/a/@href')

    title_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="title"]/a/@title')
    author_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[not(@class)]/a/text()')
    author_url_ls = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[not(@class)]/a/@href')
    last_response = s.xpath('//table[@class="olt"]//tr[not(contains(@class,"th"))]//td[@class="time"]/text()')

    df = pd.DataFrame({'url':url_ls,'title':title_ls,'author':author_ls,'author_url':author_url_ls,'last_response':last_response})
    
    return df

In [28]:
df_ls = []

In [37]:
for idx in tqdm(range(439,500)):
    start = idx*25
    url = f'https://www.douban.com/group/shanghaizufang/discussion?start={start}'
    df = get_topics(url)
    if len(df)>0:
        df_ls.append(df)
        time.sleep(random.choice(range(1,4)))
    else:
        print(idx)
        break

100%|██████████| 61/61 [02:37<00:00,  2.58s/it]


In [38]:
len(df_ls)

500

In [46]:
df_res = pd.concat(df_ls)

# 去重
df_res.drop_duplicates(inplace=True)

# 同一个帖子取最新一条回复对应的记录

df_res = df_res.sort_values(['last_response'],ascending=False).groupby(['url','title','author','author_url']).first().reset_index()

df_res.sort_values(['last_response'],ascending=False,inplace=True)

In [47]:
df_res.shape

(12315, 5)

In [65]:
# 保存为json
js = df_res.to_json(orient='records')

with open('zf_topics.json','w') as f:
    json.dump(js,f)

# 一些数据统计

### 1.共有6060人发帖

In [69]:
df_res['author_url'].nunique()

6060

### 2.按发帖个数对用户进行排行，靠前的肯定是中介或者二房东

In [71]:
df_res.groupby('author_url')['url'].nunique().sort_values(ascending=False)

author_url
https://www.douban.com/people/217366446/    69
https://www.douban.com/people/140889977/    44
https://www.douban.com/people/194720754/    40
https://www.douban.com/people/234444399/    40
https://www.douban.com/people/233813711/    38
                                            ..
https://www.douban.com/people/225835274/     1
https://www.douban.com/people/225825787/     1
https://www.douban.com/people/225824336/     1
https://www.douban.com/people/225823797/     1
https://www.douban.com/people/101635255/     1
Name: url, Length: 6060, dtype: int64

### 3.发帖大于等于3个的用户有1179人，占总用户的20%

In [76]:
pv_stats = df_res.groupby('author_url')['url'].nunique().sort_values(ascending=False)
sum(pv_stats>=3)

1179

### 4.这20%的用户发了6446帖子，占总数的52%

In [78]:
print(sum(pv_stats[pv_stats>=3]),sum(pv_stats[pv_stats>=3])/sum(pv_stats))

6446 0.5234267153877386


---

**20%的中介贡献了超过一半的内容，怪不得大家说现在豆瓣都被中介攻陷了**

# 过滤一下

In [48]:
import re

In [49]:
flag_zhuanzu =  df_res['title'].apply(lambda x:'转' in x)

flag_subway = df_res['title'].apply(lambda x:re.search('虹桥路|吴中路|漕河泾开发区|姚虹路',x))


In [50]:
df_res[flag_zhuanzu&flag_subway]

,url,title,author,author_url,last_response
7430,https://www.douban.com/group/topic/216853903/,个人转租，闵行徐汇湘府花园吴中路姚虹路宋园路,Shieh,https://www.douban.com/people/169185158/,03-24 21:35
11960,https://www.douban.com/group/topic/217164114/,转租2600大主卧/徐汇区/9号桂林路/15号线吴中路地铁站,随风踏云,https://www.douban.com/people/152826967/,03-24 20:34
2831,https://www.douban.com/group/topic/215084766/,【个人转租】9号线漕河泾开发区站虹兴苑小区 朝南大卧室只租女生,于二花,https://www.douban.com/people/140592316/,03-24 20:11
3948,https://www.douban.com/group/topic/215950506/,【个人转租】3、4、10号线虹桥路，仅限女生！,豆油(1),https://www.douban.com/people/only-feel/,03-24 19:43
8622,https://www.douban.com/group/topic/216932044/,个人转租万源新城 9号线合川路、漕河泾开发区近字节跳动,Stella🌟,https://www.douban.com/people/221649503/,03-24 09:30
1953,https://www.douban.com/group/topic/213904892/,9号线 桂林路地铁站 15号线 吴中路地铁站 田林 桂林苑 钦州北路888弄 次卧【转租】,江湖骗子,https://www.douban.com/people/146119171/,03-23 19:53
4592,https://www.douban.com/group/topic/216255954/,转租合川路地铁站附近近字节，漕河泾开发区可短租4个月,blowww~,https://www.douban.com/people/225348940/,03-23 19:49
5535,https://www.douban.com/group/topic/216585832/,闵行区吴中路朗诗寓转租,豆友226196802,https://www.douban.com/people/226196802/,03-23 17:52
2519,https://www.douban.com/group/topic/214808884/,转·近吴中路虹桥路中山西路一室一厅3/4/9/10号线超方便（非中介）,Karenina,https://www.douban.com/people/188209507/,03-23 15:22
4399,https://www.douban.com/group/topic/216197062/,转租 漕河泾开发区对面，中友嘉园小区，12号线虹梅路站，独卫2650,离开上海倒计时,https://www.douban.com/people/234553789/,03-23 13:56


In [79]:
browser.quit()